In [1]:
import pandas as pd
import itertools
from sklearn import preprocessing
import numpy as np
import math
import datetime

pd.set_option('display.max_columns', 500)

In [2]:
### import the csvs, merge on 'id' column, export to csv
data_test = pd.read_csv('watertabletestingvalues.csv')
data_values = pd.read_csv('watertabletrainingvalues.csv')
data_labels = pd.read_csv('watertabletraininglabels.csv')
data_values.merge(data_labels[['id','status_group']],on='id').to_csv('waterabletrainingset.csv')

In [3]:
data = pd.read_csv('waterabletrainingset.csv').drop(columns=['Unnamed: 0'])
data

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322e+00,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466e+00,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329e+00,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-1.115530e+01,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359e+00,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587e+00,Tajiri,0,Pangani,Moa/Mwereme,Tanga,4,8,Mkinga,Moa,1,True,GeoData Consultants Ltd,VWC,Zingibali,True,2009,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,functional
6,19816,0.0,2012-10-01,Dwsp,0,DWSP,33.362410,-3.766365e+00,Kwa Ngomho,0,Internal,Ishinabulandi,Shinyanga,17,3,Shinyanga Rural,Samuye,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,swn 80,swn 80,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,non functional
7,54551,0.0,2012-10-09,Rwssp,0,DWE,32.620617,-4.226198e+00,Tushirikiane,0,Lake Tanganyika,Nyawishi Center,Shinyanga,17,3,Kahama,Chambo,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,non functional
8,53934,0.0,2012-11-03,Wateraid,0,Water Aid,32.711100,-5.146712e+00,Kwa Ramadhan Musa,0,Lake Tanganyika,Imalauduki,Tabora,14,6,Tabora Urban,Itetemia,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump,non functional
9,46144,0.0,2011-08-03,Isingiro Ho,0,Artisan,30.626991,-1.257051e+00,Kwapeto,0,Lake Victoria,Mkonomre,Kagera,18,1,Karagwe,Kaisho,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallo

In [4]:
### IGNORE THIS CELL, 
category = 'lga'

print(data[category].value_counts())
print(data[category].nunique())

for thing in data_test[category].unique():
    if thing not in data[category].unique():
        print(thing+ ' in test not train')

for thing in data[category].unique():
    if thing not in data_test[category].unique():
        print(thing+ ' in train not test')

Njombe              2503
Arusha Rural        1252
Moshi Rural         1251
Bariadi             1177
Rungwe              1106
Kilosa              1094
Kasulu              1047
Mbozi               1034
Meru                1009
Bagamoyo             997
Singida Rural        995
Kilombero            959
Same                 877
Kibondo              874
Kyela                859
Kahama               836
Kigoma Rural         824
Magu                 824
Maswa                809
Karagwe              771
Mbinga               750
Iringa Rural         728
Serengeti            716
Lushoto              694
Namtumbo             694
Songea Rural         693
Mpanda               679
Mvomero              671
Ngara                669
Ulanga               665
                    ... 
Ileje                231
Bahi                 224
Kisarawe             223
Temeke               215
Rorya                210
Tarime               209
Ngorongoro           201
Kiteto               193
Shinyanga Urban      191


In [5]:
# Calculate well age
data['date_recorded_dt'] = pd.to_datetime(data['date_recorded'],format='%Y-%m-%d')
average_construction_year = data['construction_year'][(data['construction_year'] != 0)].mean()
data['age'] = (data['date_recorded_dt'].dt.year + data['date_recorded_dt'].dt.dayofyear/365) - data['construction_year'].replace(0,average_construction_year)


In [6]:
# In this dataset, null values are also written as 0. 
# function that standardizes the data ignoring the 0 values

def zero_isnan_standardize(data_series):
    
    average_value = data_series[(data_series != 0)].mean()
    X = data_series.replace(0,math.nan).values
    Xz = (X - np.nanmean(X, axis=0))/np.nanstd(X, axis=0)
    Xz[np.isnan(Xz)] = 0
    
    return Xz

In [7]:
### Function to binarize data
def binarize_strings(data_series):
    encoder = preprocessing.LabelBinarizer(sparse_output=False)
    status_array = encoder.fit_transform(data_series)
    
    return pd.DataFrame(columns=encoder.classes_,data=status_array)

In [8]:
# boolean mapping for data with NaN values
def map_bool(data_series):
    mapping = {
        True: 1,
        #replace NaN with average
        math.nan: data_series[(data_series != math.nan)].mean(),
        False: 0
    }
    return data_series.map(mapping).values

In [9]:
### Standardize Numerical Data
#list of columns to standardize with 0 as null values
standardize_columns = ['amount_tsh', # Total static head
                       'gps_height', # Altitude of the well
                       'longitude','latitude', # For approximate locations
                       'population' # Population around the well
                      ]

scaler = preprocessing.StandardScaler()

#new dataframe
data_standard = pd.DataFrame()

for column in standardize_columns:
    
    data_standard[column+'_standard'] = zero_isnan_standardize(data[column])

# standardize regular data
data_standard['age_standard'] = scaler.fit_transform(data['age'].values.reshape(-1,1))

In [10]:
bool_columns = ['permit', #If the waterpoint is permitted
                'public_meeting'
               ]
data_bool = pd.DataFrame()
for column in bool_columns:
    data_bool[column+'_mapped'] = map_bool(data[column])

In [11]:
### Binarize Text Data
binarize_columns = ['extraction_type', #experiment with extraction_type_group or extraction_type_class
                    'lga', #for more precise region
                    'management_group', #who manages well
                    'quality_group', #quality of water, experiment with water_quality
                    'payment_type', #how users pay
                    'quantity', #experiment with quantity_group
                    'source_type', #experiment with source or source_class
                    'waterpoint_type' #experiment with waterpoint type group
                   ]
data_binarize = pd.DataFrame()
for column in binarize_columns:
    data_binarize = pd.concat([data_binarize,binarize_strings(data[column])],axis=1)

In [12]:
### Final Processed DataFrame
data_processed = pd.concat([data[['id','status_group']],data_standard,data_bool,data_binarize],axis=1)
data_processed.to_csv('waterabletrainingsetprocessed.csv')
data_processed

,id,status_group,amount_tsh_standard,gps_height_standard,longitude_standard,latitude_standard,population_standard,age_standard,permit_mapped,public_meeting_mapped,afridev,cemo,climax,gravity,india mark ii,india mark iii,ksb,mono,nira/tanira,other,other - mkulima/shinyanga,other - play pump,other - rope pump,other - swn 81,submersible,swn 80,walimi,windmill,Arusha Rural,Arusha Urban,Babati,Bagamoyo,Bahi,Bariadi,Biharamulo,Bukoba Rural,Bukoba Urban,Bukombe,Bunda,Chamwino,Chato,Chunya,Dodoma Urban,Geita,Hai,Hanang,Handeni,Igunga,Ilala,Ileje,Ilemela,Iramba,Iringa Rural,Kahama,Karagwe,Karatu,Kasulu,Kibaha,Kibondo,Kigoma Rural,Kigoma Urban,Kilindi,Kilolo,Kilombero,Kilosa,Kilwa,Kinondoni,Kisarawe,Kishapu,Kiteto,Kondoa,Kongwa,Korogwe,Kwimba,Kyela,Lindi Rural,Lindi Urban,Liwale,Longido,Ludewa,Lushoto,Mafia,Magu,Makete,Manyoni,Masasi,Maswa,Mbarali,Mbeya Rural,Mbinga,Mbozi,Mbulu,Meatu,Meru,Misenyi,Missungwi,Mkinga,Mkuranga,Monduli,Morogoro Rural,Morogoro Urban,Moshi Rural,Moshi Urban,Mpanda,Mpwapwa,Mtwara Rural,Mtwara Urban,Mufindi,Muheza,Muleba,Musoma Rural,Mvomero,Mwanga,Nachingwea,Namtumbo,Nanyumbu,Newala,Ngara,Ngorongoro,Njombe,Nkasi,Nyamagana,Nzega,Pangani,Rombo,Rorya,Ruangwa,Rufiji,Rungwe,Same,Sengerema,Serengeti,Shinyanga Rural,Shinyanga Urban,Siha,Sikonge,Simanjiro,Singida Rural,Singida Urban,Songea Rural,Songea Urban,Sumbawanga Rural,Sumbawanga Urban,Tabora Urban,Tandahimba,Tanga,Tarime,Temeke,Tunduru,Ukerewe,Ulanga,Urambo,Uyui,commercial,other,parastatal,unknown,user-group,colored,fluoride,good,milky,salty,unknown,annually,monthly,never pay,on failure,other,per bucket,unknown,dry,enough,insufficient,seasonal,unknown,borehole,dam,other,rainwater harvesting,river/lake,shallow well,spring,cattle trough,communal standpipe,communal standpipe multiple,dam,hand pump,improved spring,other
0,69572,functional,0.912825,0.605884,-0.081144,-1.408791,-0.304752,-0.319943,0.00000,1.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
1,8776,functional,0.000000,0.620576,-0.172932,1.207934,-0.001925,-1.213851,1.00000,0.909838,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,34310,functional,-0.191776,-0.543395,0.886320,0.639751,-0.055053,-1.117212,1.00000,1.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
3,67743,non functional,0.000000,-1.233941,1.279621,-1.849720,-0.395068,1.154068,1.00000,1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,19728,functional,0.000000,0.000000,-1.541311,1.317271,0.000000,-0.070572,1.00000,1.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,